In [1]:
import os
import time
import importlib
import tqdm

import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

import catboost
from catboost import CatBoostClassifier
import pandas as pd

import common
importlib.reload(common)
from common import get_technical_indicators_to_predict

# Prepare API

In [2]:
key = 'W79HC61KBLPCXW6C'

In [3]:
ti = TechIndicators(key, output_format='pandas')
ts = TimeSeries(key, output_format='pandas')

# Get S&P 500 companies' symbols

In [4]:
with open('../sp_500_symbols.txt', 'r') as f:
    sp_500_symbols = f.read().split('\n')[:-1]

In [5]:
print(len(sp_500_symbols))
print(sp_500_symbols[:5])

505
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [6]:
sp_500_symbols_filtered = list(filter(lambda x: x not in ['BF.B', 'CBS'], sp_500_symbols))
print(len(sp_500_symbols_filtered))

503


# Identify stocks to buy 

In [7]:
classifier = CatBoostClassifier()
models_path = 'models'
model_path = os.path.join(models_path, 'model_v4')
classifier.load_model(model_path)

In [8]:
tech_ind_df_offset = pd.read_csv(os.path.join(models_path, 'model_v4_offset'), header = None, 
                                 index_col = 0, squeeze = True).drop('target')
tech_ind_df_scale = pd.read_csv(os.path.join(models_path, 'model_v4_scale'), header = None, 
                                index_col = 0, squeeze = True).drop('target')

In [9]:
tech_ind_df_offset

0
rsi_7         52.791121
cci_7          9.330885
slowk         55.274583
aroon_down    44.287380
cci_14        13.820388
rsi_14        52.628647
aroon_up      54.386452
slowd         55.277047
adx_7         33.577770
dema_13       53.700543
sma_26        53.387167
mom_8          0.202034
mom_15         0.377887
mfi_7         52.502343
adx_14        24.181494
Name: 1, dtype: float64

In [10]:
tech_ind_df_scale

0
rsi_7          16.665293
cci_7         100.020979
slowk          27.869879
aroon_down     36.120299
cci_14        108.348189
rsi_14         11.872438
aroon_up       36.382896
slowd          27.033350
adx_7          12.749709
dema_13        96.847344
sma_26         95.905038
mom_8           5.749648
mom_15          7.776611
mfi_7          22.034706
adx_14          9.557651
Name: 1, dtype: float64

In [11]:
classifier.feature_names_

['rsi_7',
 'cci_7',
 'slowk',
 'aroon_down',
 'cci_14',
 'rsi_14',
 'aroon_up',
 'slowd',
 'adx_7',
 'dema_13',
 'sma_26',
 'mom_8',
 'mom_15',
 'mfi_7',
 'adx_14']

In [12]:
def identify_stocks_to_buy_with_model(ti, symbols, model, tech_ind_df_offset, tech_ind_df_scale):
    should_process = True
    first_stock_to_process = None
#     first_stock_to_process = 'CBS'
    
    stocks_to_buy_ti = []
    stocks_to_buy = []
    predictions = []
    for stock in tqdm.tqdm(symbols):
        start = time.time()
        print('Stock: {}'.format(stock))
        
        if not should_process or stock == 'BF.B':
            if stock == first_stock_to_process:
                should_process = True
            print('Cannot process')
            continue
            
        try:
            tech_ind = get_technical_indicators_to_predict(ti, stock)
        except:
            print('Cannot process coz of API')
            continue
        data_x = (tech_ind.iloc[-1:] - tech_ind_df_offset) / tech_ind_df_scale
#         print(data_x)
        
        pred = classifier.predict_proba(data_x)[0][1]
        print('Prediction: {}'.format(pred))
        
        should_buy = True if pred > 0.9 else False
        if should_buy:
            stocks_to_buy_ti.append(data_x)
            stocks_to_buy.append(stock)
            predictions.append(pred)
            
        end = time.time()
        process_time = end - start
        print('Time: {}'.format(process_time))
        time.sleep(max(31 - process_time, 1))  # API calls frequency restrictions
        
    return stocks_to_buy_ti, stocks_to_buy, predictions

In [ ]:
stocks_to_buy_ti, stocks_to_buy, predictions = identify_stocks_to_buy_with_model(
    ti, 
    sp_500_symbols_filtered, 
    classifier, 
    tech_ind_df_offset, 
    tech_ind_df_scale
)

  0%|          | 0/503 [00:00<?, ?it/s]

Stock: MMM
Prediction: 0.9999281894148857
Time: 18.08909559249878


  0%|          | 1/503 [00:31<4:19:28, 31.01s/it]

Stock: AOS
Prediction: 2.1260268955806566e-05
Time: 17.32096219062805


  0%|          | 2/503 [01:02<4:18:58, 31.01s/it]

...

Stock: KSU
Prediction: 0.0014632613733989732
Time: 17.39343571662903


 53%|█████▎    | 265/503 [2:15:17<2:03:01, 31.01s/it]

Stock: K
Prediction: 0.997909021009285
Time: 18.564700603485107


 53%|█████▎    | 266/503 [2:15:48<2:02:30, 31.01s/it]

Stock: KEY


In [ ]:
stock_pred = sorted(zip(stocks_to_buy, predictions), key=lambda x: x[1], reverse=True)
for s, p in stock_pred:
    print('Stock: {} | Pred: {}'.format(s, p))